In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Load training data
file_path = r'C:\...\Plot_Extract_Data+Estimated_MH.csv'
df = pd.read_csv(file_path)

# Filter the data
excluded_values = [1846]            
df = df[~df['UniqueID'].isin(excluded_values)]

# Define predictors and target variable
predictors = [
    'B3_Jul', 'B3_Jun', 'B5_Jul', 'B5_Jun', 'CAB_May', 'GLCM_VV_dent_Jan', 
    'MCARI_Aug', 'Predicted_Height_A', 'TCARI_Jul', 'TCARI_Jun', 'sarRI2_Feb', 'sarSqDI_Sep'
]
target_variable = 'Dw_All_Mg'

# Train the RandomForestRegressor model with specified hyperparameters
model = RandomForestRegressor(n_estimators=100, max_features=10, random_state=7).fit(df[predictors], df[target_variable])

# Extract feature importances from the model
feature_importances = model.feature_importances_

# Create a DataFrame for easier viewing
importances_df = pd.DataFrame({
    'Feature': predictors,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Display the importances
print(importances_df)

In [ ]:
import numpy as np
import rasterio
import warnings

def predict_sd(input_image_path, model, predictors, output_folder_path, output_resolution=10):
    with rasterio.open(input_image_path) as src:
        # Adjust the transform for the output resolution if needed
        transform = src.transform * src.transform.scale(
            (src.width / src.width * output_resolution) / 10,
            (src.height / src.height * output_resolution) / 10
        )
        
        # Update metadata for the output raster, specifically for SD
        meta = src.meta.copy()
        meta.update(count=1, dtype='float32', transform=transform)
        
        sd_array = np.full((src.height, src.width), np.nan, dtype='float32')  # Initialize SD array
        
        for window in src.block_windows(1):
            # Stack the required bands for prediction
            data = np.stack([src.read(i+1, window=window[1], masked=True) for i, band in enumerate(src.descriptions) if band in predictors], axis=-1)
            valid_mask = ~np.isnan(data).any(axis=-1)
            
            # Create a DataFrame for prediction to ensure feature names match
            valid_data = pd.DataFrame(data[valid_mask].reshape(-1, len(predictors)), columns=predictors)
            
            if not valid_data.empty:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    # Predict using all trees to calculate SD
                    all_predictions = np.stack([tree.predict(valid_data) for tree in model.estimators_], axis=0)
                    sd = np.std(all_predictions, axis=0)  # Calculate SD
                
                # Fill the SD array
                sd_array[window[1].row_off:window[1].row_off + window[1].height, window[1].col_off:window[1].col_off + window[1].width][valid_mask] = sd

                
        # Save the standard deviations as a single-band GeoTIFF
        with rasterio.open(output_folder_path, 'w', **meta) as dst:
            dst.write(sd_array, 1)
            dst.set_band_description(1, 'AGB_Prediction_SD')

# Set parameters for the prediction function
input_image_path = r"C:\Uni\EGM704\GEE\Exports\Final Images\Input_Raster.tif"
output_file_path = r"C:\...\SD_Map.tif"

# Execute the prediction function to only produce and save the SD map
predict_sd(input_image_path, model, predictors, output_folder_path)


In [ ]:
#Compare with map of AGB

def calculate_agb_statistics(agb_file_path, sd_file_path):
    # Load the AGB map
    with rasterio.open(agb_file_path) as agb_src:
        agb_array = agb_src.read(1, masked=True)
    
    # Load the standard deviation (SD) map
    with rasterio.open(sd_file_path) as sd_src:
        sd_array = sd_src.read(1, masked=True)
    
    # Ensure that both arrays cover the same spatial extent and have the same dimensions
    assert agb_array.shape == sd_array.shape, "AGB and SD arrays must have the same dimensions"
    
    # Calculate valid (non-NaN) values for AGB and SD
    valid_agb = agb_array.compressed()  # Removes masked values, effectively ignoring NaNs
    valid_sd = sd_array.compressed()
    
    # Calculate the average AGB
    mean_agb = np.mean(valid_agb)
    
    # Calculate the average standard deviation (mean uncertainty)
    mean_sd = np.mean(valid_sd)
    
    # Calculate the minimum and maximum standard deviation
    min_sd = np.min(valid_sd)
    max_sd = np.max(valid_sd)
    
    # Calculate the percentage of mean uncertainty with respect to mean predicted biomass
    percentage_mean_uncertainty = (mean_sd / mean_agb) * 100
    
    # Print the statistics
    print(f"Average AGB: {mean_agb} Mg/ha")
    print(f"Average SD (Uncertainty): {mean_sd} Mg/ha")
    print(f"Minimum SD (Uncertainty): {min_sd} Mg/ha")
    print(f"Maximum SD (Uncertainty): {max_sd} Mg/ha")
    print(f"Range of SD (Uncertainty): {min_sd} to {max_sd} Mg/ha")
    print(f"Percentage of Mean Uncertainty with respect to Mean Predicted Biomass: {percentage_mean_uncertainty}%")

# Paths to your AGB and SD files
agb_file_path = r"C:\...\Estimated_AGB_Map.tif"
sd_file_path = r"C:\...\SD_Map.tif"

# Execute the function
calculate_agb_statistics(agb_file_path, sd_file_path)
